In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE72625"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE72625"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE72625.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE72625.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE72625.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gastrointestinal symptoms and pathology in patients with Common variable immunodeficiency"
!Series_summary	"Based on the findings of increased IEL in duodenal biopsies in CVID, an overlap with celiac disease has been suggested. In the present study, increased IEL, in particular in the pars descendens of the duodenum, was one of the most frequent histopathological finding. We therefore examined the gene expression profile in pars descendens of duodenum in CVID patients with increased IEL (n=12, IEL mean 34 [range 22-56] IEL/100 EC), CVID with normal levels of IEL (n=8), celiac disease (n=10, Marsh grade 3a or above) and healthy controls (n=17) by gene expression microarray"
!Series_overall_design	"GI histopathological findings in 53 CVID patients that underwent upper and lower endoscopic examination were addressed. For the microarray analysis 20 CVID (8 CVID_normal and 12 CVID with incresed IEL), 10 patients with celiac diseases and 17 healthy cont

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np
from typing import Optional, Callable, Dict, Any
import json

# Check if the cohort has gene expression data based on background information
# This seems to contain gene expression microarray data from duodenal biopsies
is_gene_available = True

# Define the trait, age, and gender rows from the sample characteristics dictionary
# Trait (disease state) is available at key 0
trait_row = 0
# Age is not available in the sample characteristics dictionary
age_row = None
# Gender is not available in the sample characteristics dictionary
gender_row = None

# Define conversion functions for the clinical variables
def convert_trait(value_str):
    """Convert trait (disease state) string to binary value (Celiac disease = 1, others = 0)"""
    if value_str is None or pd.isna(value_str):
        return None
    
    # Extract the value after the colon if present
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip().lower()
    else:
        value = value_str.strip().lower()
    
    # Convert to binary based on Celiac Disease presence
    if 'celiac disease' in value:
        return 1
    elif 'cvid' in value or 'healthy control' in value:
        return 0
    else:
        return None

def convert_age(value_str):
    """Convert age string to continuous value (not used as age is not available)"""
    return None

def convert_gender(value_str):
    """Convert gender string to binary value (not used as gender is not available)"""
    return None

# Check trait data availability
is_trait_available = trait_row is not None

# Save metadata for initial filtering
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if is_trait_available:
    # Load the clinical data
    clinical_data = pd.DataFrame(
        {0: ['disease state: CVID with increased intraepithelial lymphocytes', 
             'disease state: CVID without increased intraepithelial lymphocytes', 
             'disease state: celiac disease', 
             'disease state: healthy controls'],
         1: ['tissue: duodenal biopsy'] * 4}
    )
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:", preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview: {0: [0.0], 1: [nan]}
Clinical data saved to ../../output/preprocess/Celiac_Disease/clinical_data/GSE72625.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Celiac_Disease/GSE72625/GSE72625_series_matrix.txt.gz


Gene data shape: (47323, 47)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the identifiers, these are Illumina microarray probe IDs (starting with ILMN_)
# These are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene mapping
# From the previous outputs, we can see that:
# - Gene expression data has index 'ID' with identifiers like ILMN_1343291
# - Gene annotation data has 'ID' column with same ILMN_ format identifiers 
# - Gene annotation data has 'Symbol' column which contains gene symbols

# 2. Get gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
print(f"Gene mapping shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to ensure consistency across the dataset
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (44837, 2)
First 5 rows of gene mapping:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}


Gene expression data shape after mapping: (21464, 47)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')
Gene expression data shape after normalization: (20259, 47)
First 10 normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Celiac_Disease/gene_data/GSE72625.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Re-load clinical data from matrix file to ensure we have the correct data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Re-extract clinical features with the properly loaded clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function
try:
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    print("Linked data first few columns:")
    print(linked_data.columns[:10])
    
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note="Dataset contains gene expression from duodenal biopsies of Celiac Disease patients, CVID patients, and healthy controls"
    )
    
    # 6. If the linked data is usable, save it as a CSV file
    if is_usable:
        print(f"Data is usable. Saving to {out_data_file}")
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
    else:
        print("Data is not usable. Not saving linked data file.")
        
except Exception as e:
    print(f"Error in data linking or processing: {e}")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True, 
        df=pd.DataFrame(),
        note=f"Error processing data: {e}"
    )
    print("Data is not usable due to processing error.")

Clinical data shape: (1, 47)
Clinical data preview:
{'GSM1866896': [0.0], 'GSM1866897': [0.0], 'GSM1866898': [0.0], 'GSM1866899': [0.0], 'GSM1866900': [0.0], 'GSM1866901': [0.0], 'GSM1866902': [0.0], 'GSM1866903': [0.0], 'GSM1866904': [0.0], 'GSM1866905': [0.0], 'GSM1866906': [0.0], 'GSM1866907': [0.0], 'GSM1866908': [0.0], 'GSM1866909': [0.0], 'GSM1866910': [0.0], 'GSM1866911': [0.0], 'GSM1866912': [0.0], 'GSM1866913': [0.0], 'GSM1866914': [0.0], 'GSM1866915': [0.0], 'GSM1866916': [1.0], 'GSM1866917': [1.0], 'GSM1866918': [1.0], 'GSM1866919': [1.0], 'GSM1866920': [1.0], 'GSM1866921': [1.0], 'GSM1866922': [1.0], 'GSM1866923': [1.0], 'GSM1866924': [1.0], 'GSM1866925': [1.0], 'GSM1866926': [0.0], 'GSM1866927': [0.0], 'GSM1866928': [0.0], 'GSM1866929': [0.0], 'GSM1866930': [0.0], 'GSM1866931': [0.0], 'GSM1866932': [0.0], 'GSM1866933': [0.0], 'GSM1866934': [0.0], 'GSM1866935': [0.0], 'GSM1866936': [0.0], 'GSM1866937': [0.0], 'GSM1866938': [0.0], 'GSM1866939': [0.0], 'GSM1866940': [0.0], 'G

Linked data shape after handling missing values: (47, 20260)
For the feature 'Celiac_Disease', the least common label is '1.0' with 10 occurrences. This represents 21.28% of the dataset.
The distribution of the feature 'Celiac_Disease' in this dataset is fine.

Data is usable. Saving to ../../output/preprocess/Celiac_Disease/GSE72625.csv
